In [3]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import PolynomialFeatures
from imblearn.over_sampling import SMOTE
import pickle
from lightgbm.sklearn import LGBMClassifier

/usr/local/lib/python2.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [4]:
data_train = pd.read_csv("cleaned_2013_14", low_memory = False);
data_test = pd.read_csv("cleaned_2015", low_memory = False);

In [5]:
data_test = data_test.dropna()
data_test.columns == data_train.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [6]:
def split_data(df, cols):
    x = df.drop(cols, axis = 1)
    y = df.paid
    return x, y

cols_to_drop_training = ['loan_status', 'paid', 'amnt', 'total_pymnt', 'term_adj', 'zip_code']
x_train_initial, y_train_initial = split_data(data_train, cols_to_drop_training)
x_test, y_test = split_data(data_test, cols_to_drop_training)

In [7]:
sm = SMOTE(random_state=1, ratio = 1.0)
x_train, y_train = sm.fit_sample(x_train_initial, y_train_initial)

In [8]:
x_train.shape, x_test.shape

((578050, 44), (346843, 44))

In [39]:
params = {}

params['LGBM__learning_rate'] = [0.001, 0.01, 0.1]
params['LGBM__boosting_type'] = ['gbdt', 'dart']
params['LGBM__max_depth'] = [5, 7, 10, 12, 15]
params['LGBM__lambda_l2'] = [0, 0.001, 0.01, 0.1]

In [40]:
# Use Pipeline instead of make_pipeline
pipe = Pipeline([('POLY', PolynomialFeatures(degree=2, include_bias=False)),
            ('LGBM',LGBMClassifier(objective = 'binary', metric = 'binary_logloss',
                                   n_jobs = 5, bagging_fraction = 0.7))])


# lgbm = pipe.fit(x_train, y_train)

# Instantiate GS
gs = GridSearchCV(pipe, params, verbose=10, cv=4)

In [41]:
# pipe.get_params().keys()

In [42]:
gs.fit(x_train, y_train)

Fitting 4 folds for each of 120 candidates, totalling 480 fits
[CV] LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__max_depth=5, LGBM__lambda_l2=0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__max_depth=5, LGBM__lambda_l2=0, score=0.559495965789, total=  56.5s
[CV] LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__max_depth=5, LGBM__lambda_l2=0 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s


[CV]  LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__max_depth=5, LGBM__lambda_l2=0, score=0.649094884854, total=  56.9s
[CV] LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__max_depth=5, LGBM__lambda_l2=0 


...

[CV]  LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=5, LGBM__lambda_l2=0.01, score=0.767783990257, total=  54.7s
[CV] LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=5, LGBM__lambda_l2=0.01 
[CV]  LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=5, LGBM__lambda_l2=0.01, score=0.678490367582, total=  54.3s
[CV] LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=5, LGBM__lambda_l2=0.01 
[CV]  LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=5, LGBM__lambda_l2=0.01, score=0.547961414969, total=  54.6s
[CV] LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=7, LGBM__lambda_l2=0.01 
[CV]  LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=7, LGBM__lambda_l2=0.01, score=0.583590517182, total=  58.4s
[CV] LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=7, LGBM__lambda_l2=0.01 
[CV]  LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=7, 

[CV]  LGBM__learning_rate=0.001, LGBM__boosting_type=dart, LGBM__max_depth=15, LGBM__lambda_l2=0.1, score=0.700876051816, total= 1.0min
[CV] LGBM__learning_rate=0.001, LGBM__boosting_type=dart, LGBM__max_depth=15, LGBM__lambda_l2=0.1 
[CV]  LGBM__learning_rate=0.001, LGBM__boosting_type=dart, LGBM__max_depth=15, LGBM__lambda_l2=0.1, score=0.681818810895, total= 1.0min
[CV] LGBM__learning_rate=0.001, LGBM__boosting_type=dart, LGBM__max_depth=15, LGBM__lambda_l2=0.1 
[CV]  LGBM__learning_rate=0.001, LGBM__boosting_type=dart, LGBM__max_depth=15, LGBM__lambda_l2=0.1, score=0.539996678477, total= 1.0min
[CV] LGBM__learning_rate=0.01, LGBM__boosting_type=dart, LGBM__max_depth=5, LGBM__lambda_l2=0.1 
[CV]  LGBM__learning_rate=0.01, LGBM__boosting_type=dart, LGBM__max_depth=5, LGBM__lambda_l2=0.1, score=0.570795909047, total=  56.6s
[CV] LGBM__learning_rate=0.01, LGBM__boosting_type=dart, LGBM__max_depth=5, LGBM__lambda_l2=0.1 
[CV]  LGBM__learning_rate=0.01, LGBM__boosting_type=dart, LGBM__ma

[CV]  LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=12, LGBM__lambda_l2=0.1, score=0.817579162976, total= 1.0min
[CV] LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=12, LGBM__lambda_l2=0.1 
[CV]  LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=12, LGBM__lambda_l2=0.1, score=0.717760462799, total= 1.0min
[CV] LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=12, LGBM__lambda_l2=0.1 
[CV]  LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=12, LGBM__lambda_l2=0.1, score=0.546362931798, total= 1.0min
[CV] LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=15, LGBM__lambda_l2=0.1 
[CV]  LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=15, LGBM__lambda_l2=0.1, score=0.564561218982, total= 1.1min
[CV] LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=15, LGBM__lambda_l2=0.1 
[CV]  LGBM__learning_rate=0.1, LGBM__boosting_type=dart, LGBM__max_depth=15,

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed: 682.1min finished


GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('POLY', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)), ('LGBM', LGBMClassifier(bagging_fraction=0.7, boosting_type='gbdt', class_weight=None,
        colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
        max_depth=-1, metric='binary_logloss', m...0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'LGBM__learning_rate': [0.001, 0.01, 0.1], 'LGBM__boosting_type': ['gbdt', 'dart'], 'LGBM__max_depth': [5, 7, 10, 12, 15], 'LGBM__lambda_l2': [0, 0.001, 0.01, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [43]:
print("Best parameters set found on training set:")
print()
print(gs.best_params_)
print()
print("Grid scores on training set:")
print()
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

Best parameters set found on training set:
()
{'LGBM__learning_rate': 0.1, 'LGBM__boosting_type': 'gbdt', 'LGBM__max_depth': 15, 'LGBM__lambda_l2': 0}
()
Grid scores on training set:
()
0.599 (+/-0.110) for {'LGBM__learning_rate': 0.001, 'LGBM__boosting_type': 'gbdt', 'LGBM__max_depth': 5, 'LGBM__lambda_l2': 0}
0.621 (+/-0.123) for {'LGBM__learning_rate': 0.001, 'LGBM__boosting_type': 'gbdt', 'LGBM__max_depth': 7, 'LGBM__lambda_l2': 0}
0.618 (+/-0.145) for {'LGBM__learning_rate': 0.001, 'LGBM__boosting_type': 'gbdt', 'LGBM__max_depth': 10, 'LGBM__lambda_l2': 0}
0.626 (+/-0.145) for {'LGBM__learning_rate': 0.001, 'LGBM__boosting_type': 'gbdt', 'LGBM__max_depth': 12, 'LGBM__lambda_l2': 0}
0.628 (+/-0.139) for {'LGBM__learning_rate': 0.001, 'LGBM__boosting_type': 'gbdt', 'LGBM__max_depth': 15, 'LGBM__lambda_l2': 0}
0.630 (+/-0.101) for {'LGBM__learning_rate': 0.01, 'LGBM__boosting_type': 'gbdt', 'LGBM__max_depth': 5, 'LGBM__lambda_l2': 0}
0.633 (+/-0.157) for {'LGBM__learning_rate': 0.01,

In [44]:
train_preds_true = {'proba' : gs.best_estimator_.predict_proba(x_train)[:, 1], 'true_val' : y_train}
test_preds_true = {'proba' : gs.best_estimator_.predict_proba(x_test)[:, 1], 'true_val' : y_test}

In [45]:
AUC_train = roc_auc_score(train_preds_true['true_val'], train_preds_true['proba'])
AUC_test = roc_auc_score(test_preds_true['true_val'], test_preds_true['proba'])

In [47]:
train_score = gs.best_estimator_.score(x_train, y_train)
test_score = gs.best_estimator_.score(x_test, y_test)

In [46]:
print('AUC Train: ', AUC_train);
print('AUC Test: ', AUC_test);

('AUC Train: ', 0.9160031874431516)
('AUC Test: ', 0.6405654177058699)


In [ ]:
# ## Predict
# train_predictions = lgbm.predict(x_train)
# test_predictions = lgbm.predict(x_test)

# ## Accuracy Score
# train_score = lgbm.score(x_train, y_train)
# test_score = lgbm.score(x_test, y_test)

# print("train accuracy: ", train_score)
# print("test accuracy: ", test_score)

# ## AUC
# train_predictions_class = np.where(train_predictions > 0.5, 1, 0)
# test_predictions_class = np.where(test_predictions > 0.5, 1, 0)

# print('Train ROC AUC: ', roc_auc_score(train_predictions_class, y_train))
# print('Test  ROC AUC: ', roc_auc_score(test_predictions_class, y_test))

In [ ]:
# from sklearn.metrics import roc_curve
# y_pred = lgbm.predict(x_test).ravel()
# fpr, tpr, thresholds = roc_curve(y_test, y_pred)

# from sklearn.metrics import auc
# auc_lgbm = auc(fpr, tpr)

# plt.figure(1)
# plt.plot([0, 1], [0, 1], 'k--')
# plt.plot(fpr, tpr, label='LGBM (area = {:.3f})'.format(auc_lgbm))
# plt.xlabel('False positive rate')
# plt.ylabel('True positive rate')
# plt.title('ROC curve')
# plt.legend(loc='best')
# plt.show()

In [ ]:
# Dump the tuned classifier with Pickle
pkl_filename = 'Tuned_LGBM.pkl'

# Open the file to save as pkl file
model_pkl = open(pkl_filename, 'wb')
pickle.dump(gs.best_estimator_, model_pkl)

# Close the pickle instances
model_pkl.close()